In [1]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account

In [2]:
from constants import *

In [3]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [4]:
load_dotenv()

True

In [5]:
mnemonic = os.getenv('mnemonic')

In [6]:
print (mnemonic)

cannon rifle wood empower fish sing address actual report hamster master sort glance vacuum build


In [7]:
def derive_wallets (mnemonic, coin, numderive):

    command = 'php derive -g --mnemonic="'+str(mnemonic)+'" --numderive='+str(numderive)+' --coin='+str(coin)+' --format=jsonpretty' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    return json.loads(output)

In [8]:
coins = {'eth':derive_wallets(mnemonic=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnemonic=mnemonic,coin=BTCTEST,numderive=3)}

In [9]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [10]:
eth_privatekey = coins['eth'][0]['privkey']
btc_privatekey = coins['btc-test'][0]['privkey']

print(json.dumps(eth_privatekey, indent=4, sort_keys=True))
print(json.dumps(btc_privatekey, indent=4, sort_keys=True))

"0xc8f2d8128cb88f7cf065a49a0c22cee301f1dc381073de40681bbb381eb4c3de"
"cVseDmPmf1hH1PaPjzWumctENcqr6dQRyh8q6zXi85TJ54V56tzc"


In [11]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cVseDmPmf1hH1PaPjzWumctENcqr6dQRyh8q6zXi85TJ54V56tzc",
            "pubkey": "021009170bf75d4f77bee4727af2922a3552bc5907904c70d63b7e6ee2c0d2ada7",
            "pubkeyhash": "c5d8f5cfa33ff5912ecf157667f86f20f78988d3",
            "xprv": "tprv8k8QYFfJxbzMhWtMiQpwGXVQvkcaGCwsMunYQv67JMLQhn7LXn7RiR5mhhK9sbpgc3RUZMkPFg7aLnVsQNuS8DRnUQKjsxLxV5iN5w4xcow",
            "xpub": "tpubDGpSgfhZ6yg2ayv9c4VXfw9XVn8WRY8mwDPKhS8Qid8oYGN7AAw1tuhdsnW9aNoLweiPJ3p29w7oVMKsF2imiAnhRoeAv4cnzPdnFgfyJRD"
        },
        {
            "address": "mzViQHWxB5gdoVCThWQ6vTxmMDca6BaVu8",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cU22r46zjcZxRCuNSu48BiHiYPS4dXsQTZd2yUhoynW8bZtZ313Z",
            "pubkey": "036a6561dadf561a7abe2a4d39c426a6a21090c9c46df0e6da2fd33119f5fce70f",
            "pubkeyh

In [12]:
def priv_key_to_account (coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [13]:
eth_account = priv_key_to_account(ETH,eth_privatekey)
btc_account = priv_key_to_account(BTCTEST,btc_privatekey)

In [14]:
print(eth_account)
print(btc_account)

<PrivateKeyTestnet: myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR>


In [15]:
def create_tx(coin,account,recipient,amount):
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        return {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
            "chainId": w3.eth.chain_id
        }
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [16]:
def send_tx (coin, account, recipient, amount):
    if coin =='eth':
        trxns_eth = create_tx(coin,account, recipient, amount)
        sign_trxns_eth = account.sign_transaction(trxns_eth)
        result = w3.eth.sendRawTransaction(sign_trxns_eth.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trxns_btctest= create_tx(coin,account,recipient,amount)
        sign_trxns_btctest = account.sign_transaction(trxns_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trxns_btctest)       
        return sign_trxns_btctest

### BTC TEST

In [17]:
# create BTC transaction
create_tx(BTCTEST,btc_account,"myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR", 0.001)

'{"unspents":[{"amount":100000,"confirmations":2,"script":"76a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac","txid":"45deefc57135a7af991b5962007e3773e2d469849166912733100d865e4f3083","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":1046279,"confirmations":2,"script":"76a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac","txid":"45deefc57135a7af991b5962007e3773e2d469849166912733100d865e4f3083","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR",100000],["myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR",1008131]]}'

In [18]:
#send BTC transaction
send_tx(BTCTEST,btc_account,'myZ5KWADGjMnpfGQHGyZ9qU5MWqchdXhXR',0.001)

'010000000283304f5e860d1033279166918469d4e273377e0062591b99afa73571c5efde45000000006b483045022100991225a366b0ad9fc10a2832db6d813bb5f4f7bc78f51db9ae1baf547e64cf440220316f7ccf918b14c468dde7e3a4f682bf0f7a0675298777af998fc319dec1750a0121021009170bf75d4f77bee4727af2922a3552bc5907904c70d63b7e6ee2c0d2ada7ffffffff83304f5e860d1033279166918469d4e273377e0062591b99afa73571c5efde45010000006a47304402207eaaec5ff8118c95d8acf6087a0b3cd584bc9ee530c369263022ec02a34603c302203ac1c946ba354413f2aa5b0ea18c908a96300045b6479a636d98cb339763b9ea0121021009170bf75d4f77bee4727af2922a3552bc5907904c70d63b7e6ee2c0d2ada7ffffffff02a0860100000000001976a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac03620f00000000001976a914c5d8f5cfa33ff5912ecf157667f86f20f78988d388ac00000000'

### ETH TEST

In [19]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [20]:
from web3 import Web3, HTTPProvider

In [41]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x4230d198Fb00d08009db7fF39CBf61346faafD0D"))

In [42]:
#checking the Block Number
w3.eth.blockNumber

0

In [43]:
w3.isConnected()

True

In [44]:
w3.eth.getBalance("0x4230d198Fb00d08009db7fF39CBf61346faafD0D")

100000000000000000000

In [47]:
create_tx(ETH,eth_account,"0xDcD4c5d621D9d991c8cab5A85Cbca70467Da23A8", 1)

{'to': '0xDcD4c5d621D9d991c8cab5A85Cbca70467Da23A8',
 'from': '0xbc5A3C2eF27F87bf969036a38BDc326307821D85',
 'value': 1,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0,
 'chainId': 1337}

In [49]:
send_tx(ETH,eth_account,"0xDcD4c5d621D9d991c8cab5A85Cbca70467Da23A8", 1)

ValueError: {'message': "sender doesn't have enough funds to send tx. The upfront cost is: 420000000000001 and the sender's account only has: 0", 'code': -32000, 'data': {'stack': "Error: sender doesn't have enough funds to send tx. The upfront cost is: 420000000000001 and the sender's account only has: 0\n    at VM.<anonymous> (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\lib\\runTx.ts:114:11)\n    at step (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:33:23)\n    at Object.next (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\resources\\static\\node\\node_modules\\ganache-core\\node_modules\\ethereumjs-vm\\dist\\runTx.js:14:53)\n    at fulfilled (C:\\Program Files\\WindowsApps\\GanacheUI_2.5.4.0_x64__5dg5pnz03psnj\\app\\res

In [50]:
w3.eth.getBalance("0xba51af165c60A32B3d23Df9B332b4A86cED4A1B9")

0